In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import geo distance computation
from geopy.distance import geodesic

In [ ]:
df = pd.read_csv('../backup/ar41_for_ulb.csv', sep=';', index_col=0)

In [ ]:
df_102 = df[df['mapped_veh_id'] == 102]

In [ ]:
# vectorized haversine function
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.
    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2 - lat1) / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2 - lon1) / 2.0) ** 2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))  # return the value in km since the earth_radius is in km

In [ ]:
data = df_102.copy()

data['timestamps_UTC'] = pd.to_datetime(data['timestamps_UTC'])
data = data.reset_index(drop=True)
data = data.set_index('timestamps_UTC')
data = data.sort_index()

print(len(data))

data["time_difference"] = data.index.to_series().diff().dt.total_seconds()

data_processed = data[data["time_difference"] > 5]

data_processed["time_difference"] = data_processed.index.to_series().diff().dt.total_seconds()
data_processed["time_difference"].iloc[0] = 0

# print(data["time_difference"])
# print(len(data["time_difference"]))

data["distance"] = haversine(data['lat'].shift(), data['lon'].shift(), data['lat'], data['lon']) * 1000
data["distance"].iloc[0] = 0

data_processed["distance"] = haversine(data_processed['lat'].shift(), data_processed['lon'].shift(), data_processed['lat'], data_processed['lon']) * 1000
data_processed["distance"].iloc[0] = 0

# print(data["distance"])
# print(len(data["distance"]))

data["speed"] = data["distance"]/data["time_difference"]
data["speed"].iloc[0] = 0

data_processed["speed"] = data_processed["distance"]/data_processed["time_difference"]
data_processed["speed"].iloc[0] = 0

ts = pd.Series(data["speed"], index=data.index)
ts_processed = pd.Series(data_processed["speed"], index=data_processed.index)

print(data["speed"].describe())
print(data_processed["speed"].describe())

plt.figure(figsize=(20,8))
plt.plot(ts)
plt.plot(ts_processed)

# Mark outliers as those being at more than 120 km/h
outliers = ts[ts > 120 / 3.6]
outliers_processed = ts_processed[ts_processed > 120 / 3.6]

print(len(outliers))
print(len(outliers_processed))

plt.plot(outliers, 'ro')
plt.plot(outliers_processed, 'go')

plt.legend(['raw', 'processed'])
plt.show()


# Plot without the outliers
plt.figure(figsize=(20,8))
plt.plot(ts[ts < 120 / 3.6])
plt.plot(ts_processed[ts_processed < 120 / 3.6])
plt.plot()

# Scatter plot of speed against oil pressure
plt.figure(figsize=(20,8))
data_processed = data_processed[data_processed["speed"] < 120 / 3.6]
data_processed = data_processed[data_processed["RS_E_OilPress_PC1"] > 10]
plt.scatter(data_processed["RS_E_OilPress_PC1"], data_processed["speed"])
plt.show()

In [ ]:
def find_trip_split_indexes(data_frame: pd.DataFrame, threshold):
    # Separate data by duration without data
    df = data_frame.copy()

    # Sort the data by timestamp
    df.sort_index(inplace=True)

    # Get the difference between consecutive timestamps
    diff = pd.Series(df.index, index=df.index).diff()

    # Get the right side indexes where the difference is bigger than 30min
    idx_left = diff[diff > pd.Timedelta(minutes=threshold)].index

    # Get the difference between consecutive timestamps, but in reverse order
    s_diff = df.index.to_series().diff(periods=-1)

    # Get the left side indexes where the difference is bigger than 30min
    idx_right = s_diff[s_diff < -pd.Timedelta(minutes=threshold)].index

    # Add the first and last indexes
    idx_left = [df.index.min()] + idx_left.tolist()
    idx_right = idx_right.tolist() + [df.index.max()]

    return [segment for segment in zip(idx_left, idx_right)]

In [ ]:
from numpy import argmax

data_split = data_processed.copy()

# data_split = data_split[data_split["speed"] > 0 / 3.6]

# Get the indexes where the trip is split
trip_split_indexes = find_trip_split_indexes(data_split, 5)

# Split the data into trips
trips = [data_split.loc[start:end] for start, end in trip_split_indexes]

print(len(trips))

### LOOK AT ONE TRIP ###
trip_number = 6

# Plot the speed of the first trip
plt.figure(figsize=(20,8))
plt.plot(trips[trip_number]["speed"])

# Plot outliers
plt.plot(trips[trip_number][trips[trip_number]["speed"] > 120 / 3.6]["speed"], 'ro')

plt.show()

# Compute max speed for each trip
max_speeds = [trip["speed"].max() for trip in trips]

# Count number of outliers for each trip
outliers_raw = [trip[trip["speed"] > 120 / 3.6] for trip in trips]

# Bar plot of max speeds
plt.figure(figsize=(20,8))
# plt.bar(range(len(max_speeds)), max_speeds)
plt.bar(range(len(outliers_raw)), [len(outlier)/len(trips[i]) for i, outlier in enumerate(outliers_raw)])
plt.show()

### START PROCESSING DATA ###
# Remove first speed of each trip
for trip in trips:
    trip.loc[0, "speed"] = 0

### END PROCESSING DATA ###

# Compute max speed for each trip
max_speeds = [trip["speed"].max() for trip in trips]

# Count number of outliers for each trip
outliers_processed = [trip[trip["speed"] > 120 / 3.6] for trip in trips]

# Bar plot of max speeds
plt.figure(figsize=(20,8))
# plt.bar(range(len(max_speeds)), max_speeds)
plt.bar(range(len(outliers_processed)), [len(outlier)/len(trips[i]) for i, outlier in enumerate(outliers_processed)])
plt.show()

# Plot the difference between the number of outliers in the raw and processed data
plt.figure(figsize=(20,8))

raw = [len(outlier)/len(trips[i]) for i, outlier in enumerate(outliers_raw)]
processed = [len(outlier)/len(trips[i]) for i, outlier in enumerate(outliers_processed)]

plt.bar(range(len(raw)), [raw[i] - processed[i] for i in range(len(raw))])
plt.show()
